In [1]:
import random
import numpy as np
import pandas as pd

random.seed(2)  # Set the seed for Python's random module
np.random.seed(3)  # Set the seed for numpy's random module

In [2]:
from cabm import cabm_model

In [7]:
model = cabm_model.ConsumerModel(1000, "config.toml", enable_ads=True, enable_pricepoint=False)

/Users/michael/Documents/github/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/Users/michael/Documents/github/consumer-abm/cabm/ad_calendar.py:78: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [8]:
num_steps = 260

for i in range(num_steps):
    model.step()

In [9]:
agent_df = model.datacollector.get_agent_vars_dataframe()

In [10]:
agent_df

Household_Size  Consumption_Rate Brand_Preference Brand_Choice  \
Step AgentID                                                                   
0    0                     2          3.674040                B            B   
     1                     2          3.023251                A            A   
     2                     1          2.286399                A            A   
     3                     1          2.714218                B            B   
     4                     3          1.937797                A            A   
...                      ...               ...              ...          ...   
259  23                    2          2.921022                A            A   
     478                   5          3.354670                A            A   
     70                    1          3.401162                A            A   
     788                   3          2.694998                A            A   
     790                   3          2.888648                A            A   

              Loyalty_Rate                             Purchase_Probabilities  \
Step AgentID                                                                    
0    0            0.853062  {'A': 0.14693841734873503, 'B': 0.853061582651...   
     1            0.967357  {'A': 0.967356814563031, 'B': 0.03264318543696...   
     2            0.854137  {'A': 0.8541374010845748, 'B': 0.1458625989154...   
     3            0.800811  {'A': 0.19918880022419982, 'B': 0.800811199775...   
     4            0.902798  {'A': 0.9027978014017092, 'B': 0.0972021985982...   
...                    ...                                                ...   
259  23           0.916178  {'A': 0.9161775983850493, 'B': 0.0838224016149...   
     478          0.940531  {'A': 0.9405312599750452, 'B': 0.0594687400249...   
     70           0.955474  {'A': 0.9811243072580981, 'B': 0.0188756927419...   
     788          0.816733  {'A': 0.924374850668936, 'B': 0.07562514933106...   
     790          0.903061  {'A': 0.903061153291781, 'B': 0.09693884670821...   

              Enable_Ads  Ad_Decay_Factor  \
Step AgentID                                
0    0              True         4.062749   
     1              True         2.813665   
     2              True         1.647448   
     3              True         2.607525   
     4              True         2.842940   
...                  ...              ...   
259  23             True         3.263116   
     478            True         2.679838   
     70             True         1.565176   
     788            True         1.025273   
     790            True         2.581013   

                                          Ad_Channel_Preference  \
Step AgentID                                                      
0    0        {'TV': 0.46701605189161477, 'Web': 0.532983948...   
     1        {'TV': 0.4364337674539175, 'Web': 0.5635662325...   
     2        {'TV': 0.4399991523712377, 'Web': 0.5600008476...   
     3        {'TV': 0.39234460890766426, 'Web': 0.607655391...   
     4        {'TV': 0.42638741681686165, 'Web': 0.573612583...   
...                                                         ...   
259  23       {'TV': 0.4623908379907088, 'Web': 0.5376091620...   
     478      {'TV': 0.3946079195788005, 'Web': 0.6053920804...   
     70       {'TV': 0.2546422901458625, 'Web': 0.7453577098...   
     788      {'TV': 0.37439056155895023, 'Web': 0.625609438...   
     790      {'TV': 0.25448706763131634, 'Web': 0.745512932...   

                                          Adstock  ...  Last_Product_Price  \
Step AgentID                                       ...                       
0    0                           {'A': 0, 'B': 0}  ...                 5.0   
     1                           {'A': 0, 'B': 0}  ...                 5.0   
     2                           {'A': 0, 'B': 0}  ...                 5.0   
     3                           {'A': 0, 'B': 0}  ...                 

In [6]:
# Prep joint calendar for plotting
joint_calendar_df = model.config.joint_calendar

In [9]:
joint_calendar_df.columns

MultiIndex([('A', 'price'),
            ('A',    'TV'),
            ('A',   'Web'),
            ('B', 'price'),
            ('B',    'TV'),
            ('B',   'Web')],
           )

In [12]:
# Chain copies of the 52 week plan together  to cover multi-year simulation (see note in TODO.txt about making this more flexible and robust)



joint_calendar_df['TV_A'] = joint_calendar_df[('A', 'TV')]
joint_calendar_df['Web_A'] = joint_calendar_df[('A', 'Web')]
joint_calendar_df['TV_B'] = joint_calendar_df[('B', 'TV')]
joint_calendar_df['Web_B'] = joint_calendar_df[('B', 'Web')]

# Create a date range starting from January 2021
date_range = pd.date_range(start='1/1/2021', periods=num_steps, freq='W')

# Repeat the joint_calendar_df to match the length of model_df
joint_calendar_df = pd.concat([joint_calendar_df]*int(len(date_range)/len(joint_calendar_df)), ignore_index=True)

# Add the date range as a new column in joint_calendar_df
joint_calendar_df['Date'] = date_range

In [13]:
agent_df.to_pickle('agent_output_A_ADS_ONLY.pkl')
joint_calendar_df.to_pickle('joint_calendar_A_ADS_ONLY.pkl')